# NBA API

In [3]:
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import teamgamelog, leaguegamefinder
import pandas as pd

## Basics

https://github.com/swar/nba_api/blob/master/docs/examples/Basics.ipynb

## Teams

In [2]:
team_id = 1610612738 # celtics https://www.nba.com/stats/team/1610612738
# previous_season = datetime.now().year - 1
# game_log = fetch_data(teamgamelog.TeamGameLog, team_id=team_id, season=previous_season)
game_log = teamgamelog.TeamGameLog(team_id=team_id, season='2024-25').get_data_frames()[0]
game_log

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612738,0022400089,"OCT 26, 2024",BOS @ DET,W,3,0,1.0,240,38,...,0.867,13,24,37,19,10,6,10,16,124
1,1610612738,0022400073,"OCT 24, 2024",BOS @ WAS,W,2,0,1.0,240,42,...,0.778,12,37,49,21,7,3,14,19,122
2,1610612738,0022400061,"OCT 22, 2024",BOS vs. NYK,W,1,0,1.0,240,48,...,0.875,11,29,40,33,6,3,3,15,132


In [4]:
g = game_log.iloc[0]
g

Team_ID        1610612738
Game_ID        0022400089
GAME_DATE    OCT 26, 2024
MATCHUP         BOS @ DET
WL                      W
W                       3
L                       0
W_PCT                 1.0
MIN                   240
FGM                    38
FGA                    90
FG_PCT              0.422
FG3M                   22
FG3A                   48
FG3_PCT             0.458
FTM                    26
FTA                    30
FT_PCT              0.867
OREB                   13
DREB                   24
REB                    37
AST                    19
STL                    10
BLK                     6
TOV                    10
PF                     16
PTS                   124
Name: 0, dtype: object

In [5]:
matchup = g['MATCHUP'].replace(' vs. ', '-vs-').replace(' @ ', '-vs-').lower()
game_id = g['Game_ID']
print(f"https://www.nba.com/game/{matchup}-{game_id}")

https://www.nba.com/game/bos-vs-det-0022400089


In [7]:
league_game_finder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable='2024-25')
games = league_game_finder.get_data_frames()[0]
games[games['GAME_ID'] == '0022400089']


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
5,22024,1610612738,BOS,Boston Celtics,0022400089,2024-10-26,BOS @ DET,W,239,124,...,0.867,13,24,37,19,10,6,10,16,6.0
12,22024,1610612765,DET,Detroit Pistons,0022400089,2024-10-26,DET vs. BOS,L,240,118,...,0.818,9,35,44,27,4,7,14,22,-6.0


## Live Data


https://github.com/swar/nba_api/blob/master/docs/examples/LiveData.ipynb

In [3]:
games = scoreboard.ScoreBoard()
# games.get_dict()
games.get_dict()['scoreboard']['games'][0]

{'gameId': '0022400087',
 'gameCode': '20241026/LACDEN',
 'gameStatus': 3,
 'gameStatusText': 'Final',
 'period': 4,
 'gameClock': '',
 'gameTimeUTC': '2024-10-26T21:00:00Z',
 'gameEt': '2024-10-26T17:00:00Z',
 'regulationPeriods': 4,
 'ifNecessary': False,
 'seriesGameNumber': '',
 'gameLabel': '',
 'gameSubLabel': '',
 'seriesText': '',
 'seriesConference': '',
 'poRoundDesc': '',
 'gameSubtype': '',
 'homeTeam': {'teamId': 1610612743,
  'teamName': 'Nuggets',
  'teamCity': 'Denver',
  'teamTricode': 'DEN',
  'wins': 0,
  'losses': 2,
  'score': 104,
  'seed': None,
  'inBonus': None,
  'timeoutsRemaining': 0,
  'periods': [{'period': 1, 'periodType': 'REGULAR', 'score': 19},
   {'period': 2, 'periodType': 'REGULAR', 'score': 23},
   {'period': 3, 'periodType': 'REGULAR', 'score': 29},
   {'period': 4, 'periodType': 'REGULAR', 'score': 33}]},
 'awayTeam': {'teamId': 1610612746,
  'teamName': 'Clippers',
  'teamCity': 'LA',
  'teamTricode': 'LAC',
  'wins': 1,
  'losses': 1,
  'score'

In [25]:
board = scoreboard.ScoreBoard()
games = board.games.get_dict()
df = pd.DataFrame(games)

# Select and rename the desired columns
df_filtered = df[['gameId', 'gameStatusText', 'gameEt']].copy()
df_filtered['homeTeamId'] = df['homeTeam'].apply(lambda x: x['teamId'])
df_filtered['homeTeamName'] = df['homeTeam'].apply(lambda x: x['teamName'])
df_filtered['homeTeamScore'] = df['homeTeam'].apply(lambda x: x['score'])
df_filtered['awayTeamId'] = df['awayTeam'].apply(lambda x: x['teamId'])
df_filtered['awayTeamName'] = df['awayTeam'].apply(lambda x: x['teamName'])
df_filtered['awayTeamScore'] = df['awayTeam'].apply(lambda x: x['score'])

# Add winning team column
df_filtered['winningTeam'] = df_filtered.apply(
    lambda row: row['homeTeamName'] if row['homeTeamScore'] > row['awayTeamScore']
    else (row['awayTeamName'] if row['awayTeamScore'] > row['homeTeamScore']
          else 'Tie' if row['gameStatusText'] == 'Final' else 'Undefined'),
    axis=1
)

# Extract the date from gameEt and rename it gameDate
df_filtered['gameEt'] = pd.to_datetime(df_filtered['gameEt']).dt.date
df_filtered.rename(columns={'gameEt': 'gameDate'}, inplace=True)

df_filtered


,gameId,gameStatusText,gameDate,homeTeamId,homeTeamName,homeTeamScore,awayTeamId,awayTeamName,awayTeamScore,winningTeam
0,0022400097,3:30 pm ET,2024-10-27,1610612754,Pacers,0,1610612755,76ers,0,Undefined
1,0022400098,6:00 pm ET,2024-10-27,1610612751,Nets,0,1610612749,Bucks,0,Undefined
2,0022400099,6:00 pm ET,2024-10-27,1610612757,Trail Blazers,0,1610612740,Pelicans,0,Undefined
3,0022400100,7:00 pm ET,2024-10-27,1610612760,Thunder,0,1610612737,Hawks,0,Undefined
4,0022400101,8:30 pm ET,2024-10-27,1610612744,Warriors,0,1610612746,Clippers,0,Undefined


In [10]:
import pandas as pd
from nba_api.live.nba.endpoints import scoreboard
from datetime import datetime

def calculate_win_probability(elo_a, elo_b):
    return 1 / (1 + 10 ** ((elo_b - elo_a) / 400))

def get_last_games():
    
    # Load ELO ratings
    elo_df = pd.read_csv('elo-2023-24.csv')
    elo_dict = dict(zip(elo_df['Team'], elo_df['Elo']))
    
    board = scoreboard.ScoreBoard()
    last_games = board.games.get_dict()
    
    games = []
    for game in last_games:
        home_team = game['homeTeam']['teamName']
        away_team = game['awayTeam']['teamName']
        
        home_elo = elo_dict.get(home_team, 1500)  # Default ELO if not found
        away_elo = elo_dict.get(away_team, 1500)  # Default ELO if not found
        
        home_win_prob = round(calculate_win_probability(home_elo, away_elo), 2)
        away_win_prob = round(1 - home_win_prob, 2)
        
        game_info = {
            'Game ID': game['gameId'],
            'Date': game['gameEt'],
            'Home Team': home_team,
            'Away Team': away_team,
            # 'Home ELO': home_elo,
            # 'Away ELO': away_elo,
            'Home Win Probability': home_win_prob,
            'Away Win Probability': away_win_prob
        }
        games.append(game_info)
    
    return pd.DataFrame(games)

get_last_games()

,Game ID,Date,Home Team,Away Team,Home Win Probability,Away Win Probability
0,0022400087,2024-10-26T17:00:00Z,Nuggets,Clippers,0.70,0.30
1,0022400088,2024-10-26T19:00:00Z,Hornets,Heat,0.21,0.79
2,0022400089,2024-10-26T19:00:00Z,Pistons,Celtics,0.07,0.93
3,0022400090,2024-10-26T19:00:00Z,Wizards,Cavaliers,0.17,0.83
4,0022400091,2024-10-26T20:00:00Z,Bulls,Thunder,0.25,0.75
5,0022400092,2024-10-26T20:00:00Z,Grizzlies,Magic,0.22,0.78
6,0022400093,2024-10-26T20:00:00Z,Timberwolves,Raptors,0.89,0.11
7,0022400094,2024-10-26T20:30:00Z,Spurs,Rockets,0.34,0.66
8,0022400095,2024-10-26T22:00:00Z,Suns,Mavericks,0.38,0.62
9,0022400096,2024-10-26T22:30:00Z,Lakers,Kings,0.54,0.46


In [11]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

ScoreBoardDate: 2024-10-26
0022400087: Clippers vs. Nuggets @ 2024-10-26 17:00:00-04:00
0022400088: Heat vs. Hornets @ 2024-10-26 19:00:00-04:00
0022400089: Celtics vs. Pistons @ 2024-10-26 19:00:00-04:00
0022400090: Cavaliers vs. Wizards @ 2024-10-26 19:00:00-04:00
0022400091: Thunder vs. Bulls @ 2024-10-26 20:00:00-04:00
0022400092: Magic vs. Grizzlies @ 2024-10-26 20:00:00-04:00
0022400093: Raptors vs. Timberwolves @ 2024-10-26 20:00:00-04:00
0022400094: Rockets vs. Spurs @ 2024-10-26 20:30:00-04:00
0022400095: Mavericks vs. Suns @ 2024-10-26 22:00:00-04:00
0022400096: Kings vs. Lakers @ 2024-10-26 22:30:00-04:00


In [27]:
# Getting Box Scores. 
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0012400045') 
# Note: home_team & away_team have the identicial data structure.
away_team = box.away_team.get_dict()['teamName']
players = box.away_team.get_dict()['players']
f = "[{name}](https://www.nba.com/stats/player/{player_id}) ({team}): {points} PTS"
for player in players:
    print(f.format(name=player['name'],player_id=player['personId'],team=away_team,points=player['statistics']['points']))

[Jake LaRavia](https://www.nba.com/stats/player/1631222) (Grizzlies): 3 PTS
[Santi Aldama](https://www.nba.com/stats/player/1630583) (Grizzlies): 3 PTS
[Brandon Clarke](https://www.nba.com/stats/player/1629634) (Grizzlies): 2 PTS
[Desmond Bane](https://www.nba.com/stats/player/1630217) (Grizzlies): 11 PTS
[Scotty Pippen Jr.](https://www.nba.com/stats/player/1630590) (Grizzlies): 3 PTS
[Luke Kennard](https://www.nba.com/stats/player/1628379) (Grizzlies): 3 PTS
[Jaylen Wells](https://www.nba.com/stats/player/1642377) (Grizzlies): 2 PTS
[Zach Edey](https://www.nba.com/stats/player/1641744) (Grizzlies): 6 PTS
[Yuki Kawamura](https://www.nba.com/stats/player/1642530) (Grizzlies): 3 PTS
[John Konchar](https://www.nba.com/stats/player/1629723) (Grizzlies): 2 PTS
[Armando Bacot](https://www.nba.com/stats/player/1630654) (Grizzlies): 0 PTS
[Jay Huff](https://www.nba.com/stats/player/1630643) (Grizzlies): 0 PTS
[GG Jackson](https://www.nba.com/stats/player/1641713) (Grizzlies): 0 PTS
[Jaren Jack

## Finding Games

https://github.com/swar/nba_api/blob/master/docs/examples/Finding%20Games.ipynb

In [8]:
from nba_api.stats.endpoints import leaguegamefinder
league_game_finder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable='2024-25')
games = league_game_finder.get_data_frames()[0]
games


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612765,DET,Detroit Pistons,0022400089,2024-10-26,DET vs. BOS,L,240,118,...,0.818,9,35,44,27,4,7,14,22,-6.0
1,22024,1610612756,PHX,Phoenix Suns,0022400095,2024-10-26,PHX vs. DAL,W,239,114,...,0.757,7,40,47,26,3,4,11,17,12.0
2,22024,1610612739,CLE,Cleveland Cavaliers,0022400090,2024-10-26,CLE @ WAS,W,239,135,...,0.676,11,37,48,27,13,10,16,27,19.0
3,22024,1610612766,CHA,Charlotte Hornets,0022400088,2024-10-26,CHA vs. MIA,L,241,106,...,0.714,15,31,46,20,6,5,17,24,-8.0
4,22024,1610612761,TOR,Toronto Raptors,0022400093,2024-10-26,TOR @ MIN,L,241,101,...,0.692,12,34,46,29,9,4,20,22,-11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,12024,1610612747,LAL,Los Angeles Lakers,0012400003,2024-10-04,LAL vs. MIN,L,241,107,...,0.913,4,34,38,28,9,12,20,20,-17.0
214,12024,1610612762,UTA,Utah Jazz,0012400002,2024-10-04,UTA vs. NZB,W,238,116,...,0.727,9,35,44,27,11,5,21,19,29.0
215,12024,1610612738,BOS,Boston Celtics,0012400001,2024-10-04,BOS @ DEN,W,240,107,...,0.765,17,27,44,29,13,8,17,24,4.0
216,12024,15020,NZB,New Zealand Breakers,0012400002,2024-10-04,NZB @ UTA,L,239,87,...,0.615,13,22,35,23,11,5,16,20,-29.0


from leaguegamefinder game, merge the two contestants with the same GAME_ID, and keep gameId, game date, homeTeam teamId and teamName and score, same for awayTeam, and winningTeam


In [23]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd
import numpy as np

# Fetch the data
league_game_finder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_nullable='2024-25')
games = league_game_finder.get_data_frames()[0]

# Function to get the last word (nickname) of the team name
def get_team_nickname(full_name):
    return full_name.split()[-1]

# Create separate dataframes for home and away teams
home_games = games[games['MATCHUP'].str.contains('vs.')].copy()
away_games = games[games['MATCHUP'].str.contains('@')].copy()

# Rename columns and extract team nicknames
home_games = home_games.rename(columns={
    'TEAM_ID': 'homeTeamId',
    'TEAM_NAME': 'homeTeamName',
    'PTS': 'homeTeamScore'
})
home_games['homeTeamName'] = home_games['homeTeamName'].apply(get_team_nickname)

away_games = away_games.rename(columns={
    'TEAM_ID': 'awayTeamId',
    'TEAM_NAME': 'awayTeamName',
    'PTS': 'awayTeamScore'
})
away_games['awayTeamName'] = away_games['awayTeamName'].apply(get_team_nickname)

# Merge home and away games on GAME_ID
merged_games = pd.merge(
    home_games[['GAME_ID', 'GAME_DATE', 'homeTeamId', 'homeTeamName', 'homeTeamScore']],
    away_games[['GAME_ID', 'awayTeamId', 'awayTeamName', 'awayTeamScore']],
    on='GAME_ID'
)

# Determine the winning team
merged_games['winningTeam'] = np.where(
    merged_games['homeTeamScore'] > merged_games['awayTeamScore'],
    merged_games['homeTeamName'],
    merged_games['awayTeamName']
)

# Rename remaining columns and reorder
final_df = merged_games.rename(columns={
    'GAME_ID': 'gameId',
    'GAME_DATE': 'gameEt'
})

# Add a placeholder for gameStatusText
final_df['gameStatusText'] = 'Final'  # Placeholder

# Reorder columns
final_df = final_df[[
    'gameId', 'gameStatusText', 'gameEt',
    'homeTeamId', 'homeTeamName', 'homeTeamScore',
    'awayTeamId', 'awayTeamName', 'awayTeamScore',
    'winningTeam'
]]

# Display the result
final_df

,gameId,gameStatusText,gameEt,homeTeamId,homeTeamName,homeTeamScore,awayTeamId,awayTeamName,awayTeamScore,winningTeam
0,0022400091,Final,2024-10-26,1610612741,Bulls,95,1610612760,Thunder,114,Thunder
1,0022400095,Final,2024-10-26,1610612756,Suns,114,1610612742,Mavericks,102,Suns
2,0022400094,Final,2024-10-26,1610612759,Spurs,109,1610612745,Rockets,106,Spurs
3,0022400090,Final,2024-10-26,1610612764,Wizards,116,1610612739,Cavaliers,135,Cavaliers
4,0022400093,Final,2024-10-26,1610612750,Timberwolves,112,1610612761,Raptors,101,Timberwolves
...,...,...,...,...,...,...,...,...,...,...
101,0012400008,Final,2024-10-06,1610612765,Pistons,120,1610612749,Bucks,87,Pistons
102,0012400009,Final,2024-10-06,1610612747,Lakers,114,1610612756,Suns,118,Suns
103,0012400004,Final,2024-10-05,1610612746,Clippers,90,1610612744,Warriors,91,Warriors
104,0012400002,Final,2024-10-04,1610612762,Jazz,116,15020,Breakers,87,Jazz


## ELO

In [ ]:
import math

def calculate_elo(rating1, rating2, k, score1, score2):
    """
    Calculate the new Elo ratings for two teams.
    
    :param rating1: Current rating of team 1
    :param rating2: Current rating of team 2
    :param k: K-factor (usually between 20 and 32)
    :param score1: Score of team 1
    :param score2: Score of team 2
    :return: Tuple of new ratings (rating1, rating2)
    """
    expected1 = 1 / (1 + math.pow(10, (rating2 - rating1) / 400))
    expected2 = 1 / (1 + math.pow(10, (rating1 - rating2) / 400))
    
    result1 = 1 if score1 > score2 else (0.5 if score1 == score2 else 0)
    result2 = 1 - result1
    
    new_rating1 = rating1 + k * (result1 - expected1)
    new_rating2 = rating2 + k * (result2 - expected2)
    
    return new_rating1, new_rating2

### Compute Elo

In [ ]:
def get_team_elo(team_id, games_df):
    """
    Calculate the current ELO rating for a team based on their game history.
    
    :param team_id: ID of the team
    :param games_df: DataFrame containing all games
    :return: Current ELO rating for the team
    """
    initial_elo = 1500
    k_factor = 20
    
    team_games = games_df[games_df['TEAM_ID'] == team_id].sort_values('GAME_DATE')
    current_elo = initial_elo
    
    for _, game in team_games.iterrows():
        opponent_abbr = game['MATCHUP'].split()[-1]
        opponent_games = games_df[(games_df['TEAM_ABBREVIATION'] == opponent_abbr) & (games_df['GAME_DATE'] < game['GAME_DATE'])]
        opponent_elo = initial_elo if opponent_games.empty else get_team_elo(opponent_games.iloc[0]['TEAM_ID'], opponent_games)
        
        team_points = game['PTS']
        opponent_points = game['PLUS_MINUS'] - team_points if game['WL'] == 'L' else team_points - game['PLUS_MINUS']
        
        new_elo, _ = calculate_elo(current_elo, opponent_elo, k_factor, team_points, opponent_points)
        current_elo = new_elo
    
    return current_elo

In [ ]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

team_id = 1610612738  # Celtics

# Fetch all games for the Celtics
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
celtics_games = gamefinder.get_data_frames()[0]

# Sort the games by date (most recent first)
celtics_games = celtics_games.sort_values('GAME_DATE', ascending=False)
celtics_games

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22024,1610612738,BOS,Boston Celtics,0022400073,2024-10-24,BOS @ WAS,W,240,122,...,0.778,12,37,49,21,7,3,14,19,20.0
1,22024,1610612738,BOS,Boston Celtics,0022400061,2024-10-22,BOS vs. NYK,W,240,132,...,0.875,11,29,40,33,6,3,3,15,23.0
2,12024,1610612738,BOS,Boston Celtics,0012400050,2024-10-15,BOS @ TOR,L,239,118,...,0.857,8,28,36,19,10,5,6,15,-1.0
3,12024,1610612738,BOS,Boston Celtics,0012400042,2024-10-13,BOS vs. TOR,W,240,115,...,0.826,18,32,50,29,7,5,13,21,4.0
4,12024,1610612738,BOS,Boston Celtics,0012400036,2024-10-12,BOS vs. PHI,W,240,139,...,0.714,16,44,60,38,9,7,13,14,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919,21983,1610612738,BOS,Boston Celtics,0028300052,1983-11-05,BOS @ WAS,W,240,120,...,0.778,5,28,33,36,7,7,16,26,NaN
3920,21983,1610612738,BOS,Boston Celtics,0028300043,1983-11-04,BOS vs. IND,W,240,121,...,0.841,11,34,45,28,8,5,22,26,NaN
3921,21983,1610612738,BOS,Boston Celtics,0028300033,1983-11-02,BOS vs. MIL,W,240,119,...,0.816,14,24,38,26,7,7,18,18,NaN
3922,21983,1610612738,BOS,Boston Celtics,0028300019,1983-10-29,BOS @ CLE,W,240,108,...,0.690,13,31,44,25,12,7,13,22,NaN


In [ ]:
celtics_team_id = 1610612738 # celtics
get_team_elo(celtics_team_id, celtics_games)

2406.791254811004

### Get elo from previous season

In [20]:
import pandas as pd
elo = pd.read_excel('elo-2023-24.xlsx',index_col=0)
elo

,Team,W,L,W%,Diff/G,Elo,Pre '24,30 Days,15 Days,Form
1,Celtics,80,21,0.792,10.7,1752,128,21,2,WWWLW
2,Mavericks,63,41,0.606,2.3,1686,228,36,−2,LLLWL
3,T-wolves,65,33,0.663,6.1,1666,149,−38,±0,LLLWL
4,Thunder,63,29,0.685,7.3,1664,150,±0,±0,LLWLL
5,Nuggets,64,30,0.681,4.3,1660,20,±0,±0,WWWLL
6,Warriors,46,37,0.554,2.3,1613,44,±0,±0,WWLWL
7,Pacers,55,45,0.550,2.5,1613,187,−19,±0,WLLLL
8,Suns,49,37,0.570,2.2,1599,55,±0,±0,WLLLL
9,Lakers,50,39,0.562,0.6,1585,15,±0,±0,LLLWL
10,Knicks,57,38,0.600,3.7,1584,11,±0,±0,LLWLL


In [26]:
# Assuming we already have the 'elo' DataFrame

# Replace 'T-wolves' with 'Timberwolves'
elo['Team'] = elo['Team'].replace('T-wolves', 'Timberwolves')

# Display the updated row to confirm the change
print("Updated row:")
print(elo[elo['Team'] == 'Timberwolves'])

# Save the updated DataFrame to a CSV file
elo.to_csv('elo-2023-24.csv', index=False)

Updated row:
           Team   W   L     W% Diff/G   Elo Pre '24 30 Days 15 Days   Form
3  Timberwolves  65  33  0.663    6.1  1666     149     −38      ±0  LLLWL


### details

In [23]:
from nba_api.stats.static import teams
import pandas as pd

# Get all teams
all_teams = teams.get_teams()

# Convert to DataFrame for easier manipulation
teams_df = pd.DataFrame(all_teams)

# Extract team names (second element of full_name)
teams_df['team_name'] = teams_df['full_name'].apply(lambda x: x.split()[-1])

# Create the teams list
teams = teams_df['team_name'].tolist()

# Sort the teams list alphabetically
teams.sort()

# Print the teams list
print("List of NBA teams:")
for team in teams:
    print(team)

print(f"\nTotal number of teams: {len(teams)}")

# # Optional: If you want to see the full DataFrame with the new 'team_name' column
# print("\nFull DataFrame:")
# print(teams_df[['id', 'full_name', 'abbreviation', 'team_name']])

List of NBA teams:
76ers
Blazers
Bucks
Bulls
Cavaliers
Celtics
Clippers
Grizzlies
Hawks
Heat
Hornets
Jazz
Kings
Knicks
Lakers
Magic
Mavericks
Nets
Nuggets
Pacers
Pelicans
Pistons
Raptors
Rockets
Spurs
Suns
Thunder
Timberwolves
Warriors
Wizards

Total number of teams: 30


In [24]:
# Assuming we already have the 'teams' list and 'elo' DataFrame

# Create a dictionary to store the results
team_presence = {}

# Check each team
for team in teams:
    # Check if the team name is in the 'Team' column of the elo DataFrame
    is_present = team in elo['Team'].values
    team_presence[team] = is_present

# Print the results
print("Team presence in ELO DataFrame:")
for team, present in team_presence.items():
    print(f"{team}: {'Present' if present else 'Not present'}")

# Count how many teams are present and not present
present_count = sum(team_presence.values())
not_present_count = len(teams) - present_count

print(f"\nSummary:")
print(f"Teams present: {present_count}")
print(f"Teams not present: {not_present_count}")

# If there are teams not present, let's list them
if not_present_count > 0:
    print("\nTeams not found in ELO DataFrame:")
    for team, present in team_presence.items():
        if not present:
            print(team)

Team presence in ELO DataFrame:
76ers: Present
Blazers: Present
Bucks: Present
Bulls: Present
Cavaliers: Present
Celtics: Present
Clippers: Present
Grizzlies: Present
Hawks: Present
Heat: Present
Hornets: Present
Jazz: Present
Kings: Present
Knicks: Present
Lakers: Present
Magic: Present
Mavericks: Present
Nets: Present
Nuggets: Present
Pacers: Present
Pelicans: Present
Pistons: Present
Raptors: Present
Rockets: Present
Spurs: Present
Suns: Present
Thunder: Present
Timberwolves: Not present
Warriors: Present
Wizards: Present

Summary:
Teams present: 29
Teams not present: 1

Teams not found in ELO DataFrame:
Timberwolves
